In [1]:
# function that takes 1 to 4 inputs and creates a different balanced datasets
# DATA0.0 360 degree images from Google dataset
# DATA0.1 360 degree images of our own

# DATA0 360 degree images with fire extinguishers (this is what was in the validation folder)
# DATA1 normal images from internet (oldouz labels )
# DATA2 deformed images with data augmentation (ari labels)

# import

In [1]:
import os
# importing shutil module  
import shutil
import random

## google drive mount

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# change work directory and check on where you are 
!ls

DATA_0	DATA_1	DATA_2


# directories

In [4]:
#change work directory
os.chdir("/content/drive/My Drive/DATASET/")

In [6]:
# var my directory
mydir = os.getcwd()
mydir

'/content/drive/My Drive/DATASET'

In [7]:
# store subfolders
DATA = os.listdir(mydir)
DATA = sorted(DATA)
DATA

['DATA_0', 'DATA_1', 'DATA_2']

In [61]:
DEFORMATION = ["Low", "None", "High"]

In [8]:
#taking the extension out

def find_all_extension_infolder(destination):
  SplitTypes=[]
  for file in os.listdir(destination):
      SplitTypes.append(file.split('.')[-1])
  set_extensions = list(set(SplitTypes))
  return (set_extensions)

In [9]:
# checking extensions out 
DATA_SIZE = []
for sub in DATA:
  set_extensions = find_all_extension_infolder(sub)
  print (set_extensions)

['jpg', 'xml']
['jpg', 'xml']
['jpg', 'xml']


In [10]:
# count the number of elements in each subfolder
# rememeber images and annotations are together so the number should be even 
def get_images_annotation_in_folders(DATA):
  """
  DATA is a vector of sub folders
  DATA_SIZE, data_dict = get_images_annotation_in_folders(DATA)
  """
  DATA_SIZE = []
  data_dict =  dict()
  for sub in DATA:
    sub_dir =  (mydir + '/' +sub)
    all_files = os.listdir(sub_dir)
    DATA_SIZE.append(len(all_files))
    annotations = [file for file in all_files if file.endswith(".xml")]
    images = [file for file in all_files if file.endswith(".jpg")]
    others = [file for file in all_files if not file in images ]
    others = [file for file in others if not file in annotations ]
    data_dict[sub] = (images, annotations)
    check = ""
    if len(annotations) != len(images):
      check = "<------------ shite"
    print ("%s contains %s files: %s images and %s annotations ,else %s %s" %(sub_dir , len(all_files), len(images), len(annotations),len(others),check))
    if len(others) > 0:
      print (others)
  return DATA_SIZE, data_dict

In [11]:
DATA_SIZE, data_dict = get_images_annotation_in_folders(DATA)

/content/drive/My Drive/DATASET/DATA_0 contains 230 files: 117 images and 113 annotations ,else 0 <------------ shite
/content/drive/My Drive/DATASET/DATA_1 contains 1030 files: 517 images and 513 annotations ,else 0 <------------ shite
/content/drive/My Drive/DATASET/DATA_2 contains 820 files: 411 images and 409 annotations ,else 0 <------------ shite


In [12]:
DATA_SIZE

[230, 1030, 820]

In [13]:
# remove unmatching images / annotations
# refresh the data_dict!
images_to_delete = []
annotations_to_delete = []
for k in data_dict.keys():
  print ("---%s----" %k)
  images, annotations = data_dict[k] # refresh the data_dict!
  print (len(images), len(annotations))
  set_a = set([ x[:-4] for x in images])
  set_b = set([ x[:-4] for x in annotations])
  images_to_delete = set_a-set_b
  annotations_to_delete = set_b-set_a
  print (len(images_to_delete),len(annotations_to_delete))

  for im in images_to_delete:
      fullpath = mydir+'/'+ k +'/'+ im + ".jpg"
      print (fullpath)
      os.remove(fullpath)

  for annotation in annotations_to_delete:
      fullpath = mydir+'/'+ k +'/'+ annotation + ".xml"
      print (fullpath)
      os.remove(fullpath)

---DATA_0----
117 113
4 0
/content/drive/My Drive/DATASET/DATA_0/GS__0058_1600878925657.jpg
/content/drive/My Drive/DATASET/DATA_0/GS__0056_1600878925657.jpg
/content/drive/My Drive/DATASET/DATA_0/GS__0057_1600878925657.jpg
/content/drive/My Drive/DATASET/DATA_0/GS__0094_1600879147284(1).jpg
---DATA_1----
517 513
5 1
/content/drive/My Drive/DATASET/DATA_1/1-images.jpg
/content/drive/My Drive/DATASET/DATA_1/2-images (2).jpg
/content/drive/My Drive/DATASET/DATA_1/1-images (5) 2.jpg
/content/drive/My Drive/DATASET/DATA_1/2-images 2.jpg
/content/drive/My Drive/DATASET/DATA_1/1-images - 2020-10-05T130842.100.jpg
/content/drive/My Drive/DATASET/DATA_1/1-images (145).xml
---DATA_2----
411 409
2 0
/content/drive/My Drive/DATASET/DATA_2/2020-10-05T130842.290-specific_def_4-size_600-zoom25.jpg
/content/drive/My Drive/DATASET/DATA_2/2020-10-05T130842.290-up-specific_def_8-size_600-zoom25.jpg


In [14]:
# check on the cleaning 
DATA_SIZE, data_dict = get_images_annotation_in_folders(DATA)

/content/drive/My Drive/DATASET/DATA_0 contains 226 files: 113 images and 113 annotations ,else 0 
/content/drive/My Drive/DATASET/DATA_1 contains 1024 files: 512 images and 512 annotations ,else 0 
/content/drive/My Drive/DATASET/DATA_2 contains 818 files: 409 images and 409 annotations ,else 0 


In [15]:
# test!
images, annotations = data_dict["DATA_2"][0],data_dict["DATA_2"][1]
images, annotations = sorted(images), sorted(annotations)
print (images[2], annotations[2])

2020-10-05T130842.160-low-zoom25.jpg 2020-10-05T130842.160-low-zoom25.xml


# dataset parameters




In [123]:
# set % of images in dataset to go for training
P = [80,80,80]# percentage of images

# number of images from each dataset
N = [p*DATA_SIZE[i]/200 for i,p in enumerate(P)]
#or set directly the number of files
#N = [100,200,100]

if (len(N) or len(P)) != len (DATA):
  print (" probabilities and number must be as many as the datasets considered")

In [124]:
# name the dataset
degree360,notdeformed, deformed = int(N[0]),int(N[1]),int(N[2])
sum_N = int(sum (N))
dataset_name = "dataset_%s-degree360_%s-notdeformed_%s-deformed_%s" %(sum_N,
                                                                     degree360,
                                                                     notdeformed, 
                                                                     deformed
                                                                     )
print (dataset_name)

dataset_827-degree360_90-notdeformed_409-deformed_327


# sample files to be moved

In [125]:
def give_sampled_files(k,sampling,mydir):
  """
  move_files, move_files_check,remaining_files = give_sampled_files(k,sampling)
  k is the dictionary key (eg DATA0)
  sampling are the indexes choosen to be sampled
  """
  # to train 
  curr_dir = mydir + '/' +k
  all_files = (os.listdir(curr_dir))
  all_images, all_annotations = data_dict[k][0],data_dict[k][1]
  all_images, all_annotations = sorted(all_images), sorted(all_annotations) #sorting is important!
  move_files = []
  move_files_check = []
  for i in sampling:
    this_im = all_images[i]
    this_ann = all_annotations[i]
    if this_im[:-4] == this_ann[:-4]: 
      move_files.append( curr_dir +'/' + this_im)
      move_files.append ( curr_dir +'/' + this_ann)
      move_files_check.append(this_im)
      move_files_check.append(this_ann)
    else:
      print ("error in the order of files!")
  remaining_files = [ (curr_dir +'/' + file) for file in all_files if not file in move_files_check]
  print ("collected %s files (TRAIN) to move" %(len(move_files)/2))
  print ("remaining %s files (TEST) " %(len(remaining_files)/2))
  return move_files,move_files_check,remaining_files

In [126]:
mydir

'/content/drive/My Drive/DATASET'

In [127]:
# create new dataset folder
main_new = mydir +"/"+dataset_name
try: 
  os.mkdir(main_new)
  print ("created dir: %s" %main_new)
except OSError as error: 
    print(error)  
    
# define subfolders
subfolders_name = ["/test/", "/train/"]
subfolders = []
for sub in subfolders_name:
  path = main_new + sub
  subfolders.append(path)
  try: 
    os.mkdir(path) 
    print ("created sub_dir: %s" %path)
  except OSError as error: 
      print(error)  

created dir: /content/drive/My Drive/DATASET/dataset_827-degree360_90-notdeformed_409-deformed_327
created sub_dir: /content/drive/My Drive/DATASET/dataset_827-degree360_90-notdeformed_409-deformed_327/test/
created sub_dir: /content/drive/My Drive/DATASET/dataset_827-degree360_90-notdeformed_409-deformed_327/train/


In [128]:
train_folder, test_folder =  subfolders[1],subfolders[0]
train_folder

'/content/drive/My Drive/DATASET/dataset_827-degree360_90-notdeformed_409-deformed_327/train/'

In [129]:
test_subdir = []
for d in DEFORMATION:
  test_sub = test_folder+d
  test_subdir.append(test_sub)
  try: 
    os.mkdir(test_sub) 
    print ("created sub_dir: %s" %test_sub)
  except OSError as error: 
      print(error)  

created sub_dir: /content/drive/My Drive/DATASET/dataset_827-degree360_90-notdeformed_409-deformed_327/test/Low
created sub_dir: /content/drive/My Drive/DATASET/dataset_827-degree360_90-notdeformed_409-deformed_327/test/None
created sub_dir: /content/drive/My Drive/DATASET/dataset_827-degree360_90-notdeformed_409-deformed_327/test/High


In [130]:
dataset_name
# define subfolders
#main_new = dataset_name
#subfolders = ["/test/", "/train/"]

'dataset_827-degree360_90-notdeformed_409-deformed_327'

In [131]:
for i in range(len(DATA)):
  k = DATA[i]
  this_test_subdir = test_subdir[i]
  
  tot_size = DATA_SIZE[i]/2
  train_N = int(N[i])

  print ("")
  print (DATA[i], tot_size)
  print (this_test_subdir)

  

  list_indexes = list(range(train_N))
  sampling = random.sample(list_indexes, k=train_N)
  move_files, move_files_check,remaining_files = give_sampled_files(k,sampling,mydir)
  if tot_size != len(move_files)/2 +len(remaining_files)/2:
    print ("error! lost some files? ")
  else:
    #current_files_train = (os.listdir(train_folder))
    for file in move_files:
      #if not file in current_files_train:
      shutil.copy2(file, train_folder) # target filename is /dst/dir/file.ext
    
    #current_files_test = (os.listdir(test_folder))
    for file in remaining_files:
      #print (file)
      shutil.copy2(file, this_test_subdir)
      #if not file in current_files_test:
      #if k == DATA[0]:
        #shutil.copy2(file, test_subdir[0]) # target filename is /dst/dir/file.ext
      #elif k == DATA[1]:
        #shutil.copy2(file, test_subdir[1]) # target filename is /dst/dir/file.ext
      #elif k == DATA[2]:
        #shutil.copy2(file, test_subdir[2]) # target filename is /dst/dir/file.ext

print ("")
print (dataset_name)




DATA_0 113.0
/content/drive/My Drive/DATASET/dataset_827-degree360_90-notdeformed_409-deformed_327/test/Low
collected 90.0 files (TRAIN) to move
remaining 23.0 files (TEST) 

DATA_1 512.0
/content/drive/My Drive/DATASET/dataset_827-degree360_90-notdeformed_409-deformed_327/test/None
collected 409.0 files (TRAIN) to move
remaining 103.0 files (TEST) 

DATA_2 409.0
/content/drive/My Drive/DATASET/dataset_827-degree360_90-notdeformed_409-deformed_327/test/High
collected 327.0 files (TRAIN) to move
remaining 82.0 files (TEST) 

dataset_827-degree360_90-notdeformed_409-deformed_327


# New Section

In [8]:
# define subfolders
#subfolders = ["/test/", "/train/"]

In [10]:
# check naming 
for i in range(len(subfolders)):
  print (mydir + subfolders[i])

/content/drive/My Drive/DATASET/test/
/content/drive/My Drive/DATASET/train/


# 